# Testing the Head

**Warning:** Before running this notebook, first make sure you understand the command you run and make sure that the robot can freely move.

**Note:** Also stop all other running Python script or notebook connected to the robot as only one connection can run at the same time.

In [1]:
%matplotlib notebook

import time
import cv2 as cv
import numpy as np

from matplotlib import pyplot as plt

In [8]:
from collections import OrderedDict
from reachy import parts

def patch_head_config(head_cls):
    # if it's 'armv7l', assume that it's the raspberry pi 4 on reachy
    head_cls.dxl_motors = OrderedDict([
        ('left_antenna', {
            'id': 30, 'offset': 26.0, 'orientation': 'direct',
            'angle-limits': [-150, 150],
        }),
        ('right_antenna', {
            'id': 31, 'offset': 90.0, 'orientation': 'direct',
            'angle-limits': [-150, 150],
        }),
    ])
    
    return head_cls

def patch_head(head_cls):
    def __init__(self, io, default_camera='right'):
        """Create new Head part."""
        parts.part.ReachyPart.__init__(self, name='head', io=io)

        #self.neck = self.create_orbita_actuator('neck', Head.orbita_config)
        self.attach_dxl_motors(parts.Head.dxl_motors)
        #self.camera = self.io.find_dual_camera(default_camera)

    head_cls.__init__ = __init__

    return head_cls

## Connect to the head

In [9]:
from reachy import Reachy, parts

In [10]:
parts.Head = patch_head_config(parts.Head)

In [11]:
parts.Head = patch_head(parts.Head)

In [15]:
reachy = Reachy(
    head=parts.Head(io='/dev/ttyUSB*')
    #head=parts.Head(io='ws'),
)

You can now connect your robot in Unity.

## Move the neck

Check that all 3 disks are present and ok.

In [ ]:
for d in reachy.head.neck.disks:
    print(d, d.temperature)

Turn compliant/stiff and check that the head is free or fixed.

In [ ]:
reachy.head.compliant = True

In [ ]:
reachy.head.compliant = False

Go to the base position.

In [ ]:
reachy.head.compliant = False
reachy.head.look_at(1, 0, 0, duration=1, wait=True)

Play some random moves.

In [ ]:
x = 0.5
y = (2 * np.random.rand() - 1) * 0.25
z = (2 * np.random.rand() - 1) * 0.25

duration = 1

reachy.head.look_at(x, y, z, duration=duration, wait=False)

real = []

t0 = time.time()
while time.time() - t0 < duration:
    real.append([d.rot_position for d in reachy.head.neck.disks])
    time.sleep(0.01)
    
plt.figure()
plt.plot(real)

## Move the antennas

Check that we have both antennas.

Turn them stiff.

In [17]:
for m in reachy.head.motors:
    m.compliant = False

Make them go to 0

In [18]:
for m in reachy.head.motors:
    m.goal_position = 0

Make them go to 45

In [19]:
for m in reachy.head.motors:
    m.goal_position = 45

(check that they both moved)

Make them go to 0 again

In [20]:
for m in reachy.head.motors:
    m.goal_position = 0

Make them follow a sinus for a few seconds.

In [21]:
t = np.linspace(0, 10, 1000)
pos = 30 * np.sin(2 * np.pi * 0.5 * t)

for p in pos:
    for m in reachy.head.motors:
        m.goal_position = p
    time.sleep(0.01)

## Access the cameras

*Note: the cameras don't seem to be working in the simulator for reachy v1.2.3. - PC*

Check the right camera.

In [ ]:
success, img = reachy.head.right_camera.read()

if success:
    plt.figure()
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

Check the left camera.

In [ ]:
success, img = reachy.head.left_camera.read()

if success:
    plt.figure()
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))